In [10]:
import leafmap
import geopandas as gpd
from geospatial_tools import DATA_DIR

## Base data

The USA polygon is base off 2018's `cb_2018_us_nation_5m` shapefile, taken from here: 
https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html

It was then processed using QGIS to keep only the contiguous states, without any islands.

The Sentinel 2 grid was taken from the kml file found here: 
https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-2/data-products

It was then processed using QGIS to keep only the grid cells that overlap with the 
contiguous states, meaning the polygon layer which is described just above.

Since our area of study is quite large, the `EPSG:5070` projection was chosen, as it
covers the whole area, introduces minimal distortion while preserving area.

In [11]:
USA_POLYGON_FILE = DATA_DIR / "usa/usa_polygon_5070.gpkg"
S2_USA_GRID_FILE = DATA_DIR / "usa/s2_grid_usa_polygon_5070.gpkg"

In [12]:
usa_polygon = gpd.read_file(USA_POLYGON_FILE)
s2_grid = gpd.read_file(S2_USA_GRID_FILE)

In [13]:
usa_polygon

,AFFGEOID,GEOID,NAME,geometry
0,0100000US,US,United States,"MULTIPOLYGON (((-2123555.702 3120381.564, -212..."


In [14]:
s2_grid

,name,folders,description,altitude,alt_mode,time_begin,time_end,time_when,geometry
0,12TUP,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1386334.944 2487548.770 0.0...
1,12TYQ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-976300.478 2523767.452 0.00...
2,12TYR,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-960099.705 2622374.255 0.00...
3,12TYN,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1008622.024 2325748.358 0.0...
4,12TYP,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-992478.385 2424861.340 0.00...
...,...,...,...,...,...,...,...,...,...
977,12TTM,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1515431.586 2304192.826 0.0...
978,12TUK,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1448525.813 2089886.667 0.0...
979,12TUQ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1371006.917 2586590.133 0.0...
980,12TUR,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1355793.563 2685354.080 0.0...


In [15]:
m = leafmap.Map(center=[40, -98], zoom=4)

# In blue, the USA polygon
m.add_gdf(usa_polygon, layer='usa')
# In red, the Sentinel 2 grid
m.add_gdf(s2_grid, layer='s2_grid', style={"color": "red"})

m

Map(center=[40, -98], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

## Creating our inference grid

From this, we want to create a grid of square polygons with which we will later on
query the [Planetary Computer](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a)
Sentinel 2 dataset and clip the selected Sentinel 2 images.

In [16]:
import time
from geospatial_tools.vector import create_vector_grid_parallel


In [17]:
grid_size = 5000
bbox = usa_polygon.total_bounds

In [18]:
start = time.time()
print("Starting processing for [create_vector_grid_parallel]")
grid_parallel = create_vector_grid_parallel(bounding_box=bbox, grid_size=grid_size, crs="EPSG:5070")
stop = time.time()
print(f"Printing len(grid_parallel) to check if grid contains same amout of polygons : {len(grid_parallel)}")
print(f"Time taken to create parallel grid: {stop - start}")

Starting processing for [create_vector_grid_parallel]
[2024-06-03 11:30:52] INFO       [MainThread][geospatial_tools.vector] Creating grid coordinates for bounding box [[-2356113.74289801   310919.59963659  2258200.17691555  3165721.6501298 ]]
[2024-06-03 11:30:52] INFO       [MainThread][geospatial_tools.vector] Creating flattened grid coordinates
[2024-06-03 11:30:52] INFO       [MainThread][geospatial_tools.vector] Number of workers used: 16
[2024-06-03 11:30:52] INFO       [MainThread][geospatial_tools.vector] Allocating polygon array for [527033] polygons
[2024-06-03 11:30:52] INFO       [MainThread][geospatial_tools.vector] Creating polygons from chunk
Printing len(grid_parallel) to check if grid contains same amout of polygons : 527033
Time taken to create parallel grid: 5.32481837272644
